In [10]:
# ============================================================
# 🏍️ TRAIN MODEL V3 – MOTORBIKE PRICE PREDICTOR (with mean stats & region)
# ============================================================
import pandas as pd
import numpy as np
import os
import joblib
import json
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import xgboost as xgb

# ============================================================
# 1️⃣ Setup paths
# ============================================================
try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    BASE_DIR = os.getcwd()

OUTPUT_DIR = os.path.abspath(os.path.join(BASE_DIR, "..", "output_datasets"))
os.makedirs(OUTPUT_DIR, exist_ok=True)

DATA_PATH = os.path.join(OUTPUT_DIR, "motorbike_final_dataset_clean.csv")
print(f"📄 DATA_PATH: {DATA_PATH}")

# ============================================================
# 2️⃣ Load data
# ============================================================
df = pd.read_csv(DATA_PATH)
print(f"✅ Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Basic cleaning
df = df[(df["Gia"] > 0) & (df["Gia"] < 5e8)]
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["Gia", "Tuoi_xe"])

# Use existing logs
if "Log_Gia" in df.columns:
    print("✅ Found existing 'Log_Gia' column.")
else:
    df["Log_Gia"] = np.log1p(df["Gia"])
    print("ℹ️ Created new 'Log_Gia' from raw price.")

if "Log_So_Km_da_di" in df.columns:
    print("✅ Found existing 'Log_So_Km_da_di' column.")
else:
    raise KeyError("❌ Không tìm thấy cột Log_So_Km_da_di trong dataset.")

# ============================================================
# 3️⃣ Feature construction
# ============================================================
df["Tuoi_xe_x_Km"] = df["Tuoi_xe"] * df["Log_So_Km_da_di"]
df["Km_moi_nam"] = np.expm1(df["Log_So_Km_da_di"]) / (df["Tuoi_xe"] + 0.1)

# ✅ Mean price per groups (statistical features)
df["Brand_mean_price"] = df["Thuong_hieu_code"].map(df.groupby("Thuong_hieu_code")["Log_Gia"].mean().to_dict())
df["Dong_mean_price"] = df["Dong_xe_code"].map(df.groupby("Dong_xe_code")["Log_Gia"].mean().to_dict())
df["Segment_mean_price"] = df["Phan_khuc_dung_tich_code"].map(df.groupby("Phan_khuc_dung_tich_code")["Log_Gia"].mean().to_dict())

print("✅ Added mean price features: Brand_mean_price, Dong_mean_price, Segment_mean_price")

# ============================================================
# 4️⃣ Feature list (bổ sung vùng miền)
# ============================================================
FEATURES = [
    "Tuoi_xe",
    "Log_So_Km_da_di",
    "Km_moi_nam",
    "Tuoi_xe_x_Km",
    "TinhTrang_x_XuatXu",
    "LoaiXe_x_PhanKhuc",
    "Thuong_hieu_code",
    "Dong_xe_code",
    "Loai_xe_code",
    "Tinh_trang_code",
    "Xuat_xu_code",
    "Phan_khuc_dung_tich_code",
    "Vung_mien_code",                 # ✅ vùng miền
    "Brand_mean_price",               # ✅ trung bình thương hiệu
    "Dong_mean_price",                # ✅ trung bình dòng xe
    "Segment_mean_price"              # ✅ trung bình phân khúc
]
TARGET = "Log_Gia"

# Check missing
missing = [c for c in FEATURES if c not in df.columns]
if missing:
    raise KeyError(f"⚠️ Dataset missing columns: {missing}")

# ============================================================
# 5️⃣ Scale numeric features
# ============================================================
scale_cols = ["Tuoi_xe", "Log_So_Km_da_di", "Km_moi_nam", "Tuoi_xe_x_Km"]
scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])
print("✅ Scaled numeric features")

# ============================================================
# 6️⃣ Train-test split
# ============================================================
X, y = df[FEATURES], df[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"📊 Train: {X_train.shape}, Test: {X_test.shape}")

# ============================================================
# 7️⃣ Model definition (fine-tuned)
# ============================================================
params = {
    "n_estimators": 1200,
    "learning_rate": 0.03,
    "max_depth": 8,
    "subsample": 0.85,
    "colsample_bytree": 0.85,
    "gamma": 0.15,
    "min_child_weight": 3,
    "reg_lambda": 1.0,
    "reg_alpha": 0.4,
    "n_jobs": -1,
    "random_state": 42,
    "objective": "reg:squarederror"
}
model = XGBRegressor(**params)

# ============================================================
# 8️⃣ Cross-validation
# ============================================================
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
r2_scores = cross_val_score(model, X, y, cv=cv, scoring="r2")
print(f"🔁 Cross-validated R²: {r2_scores.mean():.3f} ± {r2_scores.std():.3f}")

# ============================================================
# 9️⃣ Train & Evaluate
# ============================================================
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_true_vnd, y_pred_vnd = np.expm1(y_test), np.expm1(y_pred)

mae = mean_absolute_error(y_true_vnd, y_pred_vnd)
rmse = np.sqrt(mean_squared_error(y_true_vnd, y_pred_vnd))
r2 = r2_score(y_true_vnd, y_pred_vnd)

print("\n📈 Final Model Performance (V3 – Extended Features)")
print(f"   MAE  = {mae:,.0f} VND")
print(f"   RMSE = {rmse:,.0f} VND")
print(f"   R²   = {r2:.3f}")

# ============================================================
# 🔍 Feature Importance
# ============================================================
xgb.plot_importance(model, importance_type="gain", max_num_features=15)
plt.title("Feature Importance (Gain)")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "feature_importance_v3.png"))
plt.close()

# ============================================================
# 💾 Save model & scaler
# ============================================================
MODEL_PATH = os.path.join(OUTPUT_DIR, "best_model_XGBoost.pkl")
SCALER_PATH = os.path.join(OUTPUT_DIR, "scaler_XGBoost.pkl")

joblib.dump(model, MODEL_PATH)
joblib.dump(scaler, SCALER_PATH)

print(f"💾 Model saved → {MODEL_PATH}")
print(f"💾 Scaler saved → {SCALER_PATH}")
print("🎯 Training finished successfully – Version 3 Enhanced!")


📄 DATA_PATH: c:\Users\User\Desktop\motorbike_project\output_datasets\motorbike_final_dataset_clean.csv
✅ Dataset loaded: 7,208 rows × 15 columns
✅ Found existing 'Log_Gia' column.
✅ Found existing 'Log_So_Km_da_di' column.
✅ Added mean price features: Brand_mean_price, Dong_mean_price, Segment_mean_price
✅ Scaled numeric features
📊 Train: (5757, 16), Test: (1440, 16)
🔁 Cross-validated R²: 0.400 ± 0.063

📈 Final Model Performance (V3 – Extended Features)
   MAE  = 9,430,795 VND
   RMSE = 21,833,939 VND
   R²   = 0.678
💾 Model saved → c:\Users\User\Desktop\motorbike_project\output_datasets\best_model_XGBoost.pkl
💾 Scaler saved → c:\Users\User\Desktop\motorbike_project\output_datasets\scaler_XGBoost.pkl
🎯 Training finished successfully – Version 3 Enhanced!


In [5]:
try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    BASE_DIR = os.getcwd()

OUTPUT_DIR = os.path.abspath(os.path.join(BASE_DIR, "..", "output_datasets"))
MAPPING_DIR = os.path.abspath(os.path.join(BASE_DIR, "..", "mappings"))
os.makedirs(OUTPUT_DIR, exist_ok=True)

DATA_PATH = os.path.join(OUTPUT_DIR, "motorbike_final_dataset_clean.csv")
print(f"📄 DATA_PATH: {DATA_PATH}")

# ============================================================
# 2️⃣ Load data
# ============================================================
df = pd.read_csv(DATA_PATH)
print(f"✅ Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(df.columns.tolist())


📄 DATA_PATH: c:\Users\User\Desktop\motorbike_project\output_datasets\motorbike_final_dataset_clean.csv
✅ Dataset loaded: 7,208 rows × 15 columns
['Gia', 'Log_Gia', 'Tuoi_xe', 'Log_So_Km_da_di', 'Km_moi_nam', 'Tuoi_xe_x_Km', 'TinhTrang_x_XuatXu', 'LoaiXe_x_PhanKhuc', 'Vung_mien_code', 'Thuong_hieu_code', 'Dong_xe_code', 'Loai_xe_code', 'Tinh_trang_code', 'Xuat_xu_code', 'Phan_khuc_dung_tich_code']
